In [1]:
from hypersurface_tf import *
from generate_h import *
from complexNN import *
import tensorflow as tf

### Prepare the dataset:

In [2]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
HS = Hypersurface(Z, f, 10000)

In [3]:
dataset = None
for patch in HS.patches:
    for subpatch in patch.patches:
        new_dataset = generate_dataset(subpatch)
        if dataset is None:
            dataset = new_dataset
        else:
            dataset = dataset.concatenate(new_dataset)

In [4]:
dataset = dataset.shuffle(50000).batch(1000)

### Build the model:

In [5]:
class KahlerPotential(tf.keras.Model):

    def __init__(self):
        super(KahlerPotential, self).__init__()
        self.layer_1 = ComplexDense(5, 20, activation=tf.square)
        self.layer_2 = ComplexDense(20, 10, activation=tf.square)
        self.g = ComplexDense(10, 10)
        
    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.g(x)
        x = tf.linalg.diag_part(tf.matmul(x, x, adjoint_b=True))
        x = tf.math.log(x)
        return x

In [6]:
model = KahlerPotential()

In [7]:
@tf.function
def volume_form(x, weights, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True)))

    factor = tf.reduce_sum(weights * volume_form)
    return volume_form / factor

In [29]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)
learning_rate = 1
epochs = 2

for epoch in range(epochs):
    for step, (points, Omega_Omegabar, weights, restriction) in enumerate(dataset):
        start_time = time.time()
        with tf.GradientTape() as tape:
            
            omega = volume_form(points, weights, restriction)
            
            y_pred = tf.math.real(weights*Omega_Omegabar)
            y_true = tf.math.real(weights*omega)
            
            loss = tf.keras.losses.mean_absolute_percentage_error(y_pred, y_true)
            loss = loss * len(points) / 100
            
        grads = tape.gradient(loss, model.trainable_weights)
        #for weight, grad in zip(model.trainable_weights, grads):
        #    weight.assign_sub(learning_rate * grad)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        print(time.time() - start_time)

        if step % 1 == 0:
            print("step %d: loss = %.4f" % (step, loss))

NotFoundError: No registered 'AssignSubVariableOp' OpKernel for 'GPU' devices compatible with node {{node AssignSubVariableOp}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: dtype=DT_COMPLEX64
	.  Registered:  device='XLA_GPU'; dtype in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, DT_INT8, DT_COMPLEX64, DT_INT64, DT_BFLOAT16, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_CPU'; dtype in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, DT_INT8, DT_COMPLEX64, DT_INT64, DT_BFLOAT16, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_GPU_JIT'; dtype in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, DT_INT8, DT_COMPLEX64, DT_INT64, DT_BFLOAT16, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_CPU_JIT'; dtype in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, DT_INT8, DT_COMPLEX64, DT_INT64, DT_BFLOAT16, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='CPU'; dtyp

In [ ]:
for step, (x, y, weights, restriction) in enumerate(dataset):
    #print(volume_form(x, weights, restriction))
    print(len(x))
#print(time.time() - start_time)